In [5]:
run instagram_poster

In [6]:
import re
import yaml
from datetime import date
from markdown2 import markdown
from mdplain import plain
from pathlib import Path

places = []
taste_labels = ["Do Not Recommend", "Something Going For It", "Good", "Phenomenal"]

def rating_to_formatting(rating, rating_labels):
    return rating_labels[rating]
def rating_html(rating, rating_labels):
    return "&nbsp;".join(
        [
            f'<span style="color: {color if rating == ix else faded_color}" aria-hidden="{"false" if rating == ix else "true"}">{label}</span>'
            for ix, (label, color) in enumerate(zip(rating_labels, rating_colors))
        ]
    )
def suffix(d):
    return "th" if 11 <= d <= 13 else {1: "st", 2: "nd", 3: "rd"}.get(d % 10, "th")
def custom_strftime(format_, t):
    return t.strftime(format_).replace("{S}", str(t.day) + suffix(t.day))
def format_visited(visited):
    return custom_strftime("{S} %B %Y", visited)
def format_blurb(md):
    return " ".join(plain(re.sub(r"\s+", " ", md.strip())).split(" ")[:50]) + "..."
    
for place_md in Path("../places").glob("*.md"):
    try: 
        slug = place_md.parts[-1][:-3]
        relative_url = f"/places/{slug}/"
        with open(place_md) as f:
            _, frontmatter, md = f.read().split("---", 2)
        meta = yaml.load(frontmatter, Loader=yaml.Loader)
        meta["url"] = relative_url
        meta["slug"] = slug
        visited = date.fromisoformat(meta["visited"])
        meta["visited_display"] = format_visited(visited)
        meta["review_age"] = (date.today() - visited).days
        meta["taste_label"] = rating_to_formatting(meta["taste"], taste_labels)
        meta["blurb"] = format_blurb(md)

        places.append(meta)

    except Exception as e:
        print(place_md.name, e)


In [7]:
place = places[76]

In [8]:
credentials_file_path = 'credentials.json'
bot = InstagramBot(credentials_file_path)
bot.login()
bot.upload_post(place)
# After logging in, you can proceed to upload posts or perform other actions
bot.close_driver()


Message: 
Stacktrace:
#0 0x638672a40dc3 <unknown>
#1 0x63867272f4e7 <unknown>
#2 0x63867277a35d <unknown>
#3 0x63867277a411 <unknown>
#4 0x6386727bd774 <unknown>
#5 0x63867279c47d <unknown>
#6 0x6386727bac29 <unknown>
#7 0x63867279c1f3 <unknown>
#8 0x63867276d28a <unknown>
#9 0x63867276dc5e <unknown>
#10 0x638672a050eb <unknown>
#11 0x638672a0903b <unknown>
#12 0x6386729f1201 <unknown>
#13 0x638672a09ba2 <unknown>
#14 0x6386729d60bf <unknown>
#15 0x638672a2ff18 <unknown>
#16 0x638672a300f0 <unknown>
#17 0x638672a3ff14 <unknown>
#18 0x7429c5494ac3 <unknown>



IndexError: list index out of range